# 文件夹结构导出工具

## 功能简介
该脚本通过递归遍历指定文件夹，将文件夹结构以表格形式保存到 Excel 文件中，并为其中的文件生成超链接，方便快速导航。

## 使用步骤
1. 运行脚本后，会弹出文件夹选择对话框。
2. 选择需要导出的文件夹。
3. 脚本会自动将文件夹结构保存为一个 Excel 文件（文件名为 `文件夹名称_文件夹结构.xlsx`），文件保存在所选文件夹内。

## 主要特点
- **文件夹和文件结构递归导出**：完整列出子文件夹及文件。
- **文件超链接**：Excel 文件中的文件条目带有超链接，点击即可打开文件。
- **清理非法字符**：确保文件夹和文件名称可以安全存储。

## 环境依赖
- Python 3.x
- 需要安装以下库：
  - `openpyxl`（用于操作 Excel 文件）
  - `tkinter`（用于弹出文件夹选择对话框）

## 使用示例
1. 将代码保存为 `export_folder_structure.py`。
2. 在终端运行：
   ```bash
   python export_folder_structure.py


In [5]:
import os
from tkinter import Tk, filedialog
from openpyxl import Workbook
from openpyxl.styles import Font

def clean_filename(filename):
    """
    清理文件名中的非法字符，确保它们可以安全地用作文件名或存储在 Excel 文件中。
    :param filename: 原始文件名
    :return: 清理后的文件名，仅保留可打印字符
    """
    try:
        return "".join(c for c in filename if c.isprintable())  # 过滤不可打印字符
    except Exception as e:
        return f"Error: {e}"  # 返回错误信息以便排查问题

def list_folder_structure_with_hyperlinks(folder_path, current_row=1, current_col=1, ws=None):
    """
    递归遍历文件夹结构，将文件和文件夹的名称及超链接写入 Excel。
    :param folder_path: 当前文件夹路径
    :param current_row: 当前写入的 Excel 行号
    :param current_col: 当前写入的 Excel 列号
    :param ws: 当前的 Excel 工作表对象
    :return: 更新后的行号，供递归调用
    """
    try:
        # 获取当前文件夹中的所有条目（文件和子文件夹）
        entries = os.listdir(folder_path)
        for entry in entries:
            entry_path = os.path.join(folder_path, entry)  # 获取完整路径
            clean_entry = clean_filename(entry)  # 清理文件名

            # 在 Excel 当前单元格写入条目名称
            cell = ws.cell(row=current_row, column=current_col, value=clean_entry)
            if os.path.isfile(entry_path):  # 如果是文件
                cell.hyperlink = entry_path  # 为文件添加超链接
                cell.font = Font(color="0000FF", underline="single")  # 超链接样式：蓝色字体+下划线
            elif os.path.isdir(entry_path):  # 如果是文件夹
                # 递归处理子文件夹，从下一行和下一列开始写入
                current_row = list_folder_structure_with_hyperlinks(entry_path, current_row + 1, current_col + 1, ws)
            
            # 每处理一个条目，当前行号自增，跳到下一行
            current_row += 1
    except Exception as e:
        # 如果读取文件夹失败，将错误信息写入 Excel
        ws.cell(row=current_row, column=current_col, value=f"读取失败: {e}")
        current_row += 1  # 错误信息占用一行
    
    return current_row  # 返回更新后的行号

def save_to_excel_with_hyperlinks(folder_path, output_file):
    """
    将指定文件夹的结构保存到 Excel 文件，并为文件添加超链接。
    :param folder_path: 需要保存的根文件夹路径
    :param output_file: 输出的 Excel 文件完整路径
    """
    # 创建新的 Excel 工作簿和工作表
    wb = Workbook()
    ws = wb.active
    ws.title = "文件夹结构"  # 设置工作表名称

    # 在第一行第一列写入根文件夹名称
    ws.cell(row=1, column=1, value=clean_filename(os.path.basename(folder_path)))

    # 从 Excel 第二行第一列开始写入文件夹结构
    list_folder_structure_with_hyperlinks(folder_path, current_row=2, current_col=1, ws=ws)

    # 保存生成的 Excel 文件
    wb.save(output_file)
    print(f"文件夹结构已保存到 Excel 文件: {output_file}")

def main():
    """
    主函数，用于运行整个程序：
    1. 用户选择文件夹。
    2. 将文件夹结构保存到 Excel 文件。
    """
    # 创建一个隐藏的 Tkinter 主窗口
    root = Tk()
    root.withdraw()  # 隐藏主窗口以只显示对话框

    # 打开文件夹选择对话框
    folder_path = filedialog.askdirectory(title="选择文件夹")
    
    if folder_path:  # 如果用户选择了文件夹
        print(f"你选择的文件夹是: {folder_path}\n")
        
        # 根据选择的文件夹名称生成 Excel 文件名
        folder_name = os.path.basename(folder_path.rstrip("/\\"))
        output_file_name = f"{folder_name}_文件夹结构.xlsx"  # 输出文件名
        output_file = os.path.join(folder_path, output_file_name)

        # 保存文件夹结构到 Excel 文件
        save_to_excel_with_hyperlinks(folder_path, output_file)
    else:  # 如果未选择任何文件夹
        print("未选择任何文件夹。")

# 如果脚本作为主程序运行，调用 main 函数
if __name__ == "__main__":
    main()

你选择的文件夹是: /Users/johnwong/Documents/00 U Course/04 文书面试

文件夹结构已保存到 Excel 文件: /Users/johnwong/Documents/00 U Course/04 文书面试/04 文书面试_文件夹结构.xlsx
